# Dataset preparation

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchvision import transforms
import src.dataloading as data
from tqdm import tqdm


# Load data

In [5]:
train_dataloader, valid_dataloader, test_dataloader = data.load_data()

# Model creation

In [6]:

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Running model on:",device)

Running model on: cuda:0


In [7]:
model = torchvision.models.resnet50(pretrained=True)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

## Freeze model weights

In [8]:
for param in model.parameters():
    param.requires_grad = False

## loss and optimizer

In [9]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

## Trainig

In [10]:
def train(n_epochs=5):
    model.train()
    for epoch in tqdm(range(n_epochs)):
        #train_loss = []
        for data_t, targets_t in train_dataloader:
            data_t = data_t.to(device)
            targets_t = targets_t.to(device)
            optimizer.zero_grad()
            # Generate predictions
            out = model(data_t)
            # Calculate loss
            loss = criterion(out, targets_t)
            #train_loss.append(loss.item())
            # Backpropagation
            loss.backward()
            # Update model parameters
            optimizer.step()
        #print("Loss: ",np.average(train_loss))

## Validation

In [11]:
def test(dataloader=test_dataloader):
    val_loss = []
    val_acc = []
    batch_loss = 0
    correct_t = 0
    total_t = 0
    with torch.no_grad():
            model.eval()
            for data_t, target_t in (dataloader):
                data_t, target_t = data_t.to(device), target_t.to(device)
                out = model(data_t)
                loss = criterion(out, target_t)

                _,pred_t = torch.max(out, dim=1)
                correct_t += torch.sum(pred_t==target_t).item()
                total_t += target_t.size(0)
            acc = 100 * correct_t/total_t
            val_acc.append(acc)
            val_loss.append(loss.item())
            print(f'Loss: {np.mean(val_loss):.4f}, accuracy: {(acc):.4f}%\n')

# Replace last fully connected layer

In [12]:
model.fc = nn.Linear(in_features=2048, out_features=120,bias=True)



# Update loss and optimizer for new model

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
model.to(device)
print()

In [12]:
train(10)
test(valid_dataloader)

100%|██████████| 10/10 [09:55<00:00, 59.55s/it]
Loss: 0.8438, accuracy: 84.4995%



# Unfreeeze last conv block

In [13]:

for param in model.layer4[2].parameters():
    param.requires_grad = True

In [14]:
train(10)
test(valid_dataloader)

100%|██████████| 10/10 [10:34<00:00, 63.45s/it]
Loss: 0.3199, accuracy: 84.5481%



# Unfreeeze second-to-last conv block

In [15]:
for param in model.layer4[1].parameters():
    param.requires_grad = True

In [16]:
train(10)
test(valid_dataloader)

100%|██████████| 10/10 [11:09<00:00, 67.00s/it]
Loss: 0.7142, accuracy: 82.3615%



# Testing on unseen data

In [17]:
test(test_dataloader)

Loss: 0.1517, accuracy: 84.0136%



# Untrained model

In [18]:
model = torchvision.models.resnet50(pretrained=False)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
criterion = nn.CrossEntropyLoss()
model.to(device)
print()

In [19]:
train(5)
test(valid_dataloader)

100%|██████████| 5/5 [17:15<00:00, 207.08s/it]
Loss: 3.9535, accuracy: 7.4344%

